In [12]:
%run ../yp_utils.py
import re

# Initial setup

In [2]:
paper_pmid = 31451498
paper_name = 'parisi_bleackley_2019' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/AAC.01097-19-sd002.xlsx', sheet_name='Log2 ratios')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5239 x 6


In [7]:
original_data.head()

,BARCODE,Gene name,DmAMP1,NaD1,NbD6,SBI6
0,YAL002W,VPS8,0.432123,-1.040648,0.972681,0.545181
1,YAL004W,NaN,-0.287130,0.207614,-0.054085,-1.223747
2,YAL005C,SSA1,0.448069,-0.211887,0.107067,-0.198521
3,YAL007C,ERP2,-1.760734,-0.352202,-0.427062,-0.004279
4,YAL008W,FUN14,-0.633826,-0.259746,-0.293921,-0.311902


In [40]:
original_data['orf'] = original_data['BARCODE'].astype(str)

In [41]:
strs = ['-1t','-2t','-1d','-2d','-1l','-1','-2','-3','-t','-d']

In [42]:
# Remove "-1", "-2" and "-1t" from orfs
for s in strs:
    original_data['orf'] = original_data['orf'].apply(lambda x: x.replace(s,''))

In [43]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [44]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'].values, to='orf')

In [45]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [BARCODE, Gene name, DmAMP1, NaD1, NbD6, SBI6, orf]
Index: []


In [46]:
original_data.set_index('orf', inplace=True)

In [47]:
original_data = original_data[['DmAMP1', 'NaD1', 'NbD6', 'SBI6']].copy()
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [48]:
original_data = original_data.groupby(original_data.index).mean()

In [49]:
original_data.shape

(4985, 4)

In [61]:
num_vals = (np.abs(original_data)>0).sum(axis=1)

In [64]:
original_data = original_data.loc[num_vals > 0,:]

In [65]:
original_data.shape

(4347, 4)

# Prepare the final dataset

In [66]:
data = original_data.copy()

In [67]:
dataset_ids = [21834,21833,21835,21836]
datasets = datasets.reindex(index=dataset_ids)

In [68]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [69]:
data.head()

dataset_id,21834,21833,21835,21836
data_type,value,value,value,value
orf,,,,
YAL002W,0.432123,-1.040648,0.972681,0.545181
YAL004W,-0.287130,0.207614,-0.054085,-1.223747
YAL005C,0.448069,-0.211887,0.107067,-0.198521
YAL007C,-1.760734,-0.352202,-0.427062,-0.004279
YAL008W,-0.633826,-0.259746,-0.293921,-0.311902


## Subset to the genes currently in SGD

In [70]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 27


In [71]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21834,21833,21835,21836
,data_type,value,value,value,value
gene_id,orf,,,,
2,YAL002W,0.432123,-1.040648,0.972681,0.545181
1863,YAL004W,-0.287130,0.207614,-0.054085,-1.223747
4,YAL005C,0.448069,-0.211887,0.107067,-0.198521
5,YAL007C,-1.760734,-0.352202,-0.427062,-0.004279
6,YAL008W,-0.633826,-0.259746,-0.293921,-0.311902


# Normalize

In [72]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [73]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [74]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,21834,21833,21835,21836,21834,21833,21835,21836
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
2,YAL002W,0.432123,-1.040648,0.972681,0.545181,0.725137,-1.065237,1.497497,0.946817
1863,YAL004W,-0.287130,0.207614,-0.054085,-1.223747,-0.037219,0.116471,-0.053232,-1.814894
4,YAL005C,0.448069,-0.211887,0.107067,-0.198521,0.742039,-0.280663,0.190157,-0.214276
5,YAL007C,-1.760734,-0.352202,-0.427062,-0.004279,-1.599132,-0.413497,-0.616540,0.088981
6,YAL008W,-0.633826,-0.259746,-0.293921,-0.311902,-0.404691,-0.325970,-0.415457,-0.391290


# Print out

In [75]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [76]:
from IO.save_data_to_db3 import *

In [77]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 31451498...
Inserting the new data...


100%|██████████| 4/4 [00:24<00:00,  6.24s/it]

Updating the data_modified_on field...
